# MNIST digits classification with TensorFlow

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
# import keras
%matplotlib inline
# import tensorflow as tf
import torch
from matplotlib import pyplot as plt
from sklearn.manifold import TSNE
from IPython.display import clear_output, display_html, HTML
import contextlib
import time
import io
import urllib
import base64
from tqdm import tqdm_notebook as tqdm
from umap import UMAP



# Look at the data

In this task we have 50000 28x28 images of digits from 0 to 9.
We will train a classifier on this data.

In [ ]:
# from util import load_mnist
X_train, y_train, X_val, y_val, X_test, y_test = load_mnist(flatten=True)

plt.figure(figsize=[6, 6])
for i in range(4):
    plt.subplot(2, 2, i + 1)
    plt.title("Label: %i" % y_train[i])
    plt.imshow(X_train[i].reshape([28, 28]), cmap='gray');

In [ ]:
n = 1000

little_X = []
for i in range(10):
    little_X.append(X_train[np.where(y_train == i)[0][: n]])
    
representation = np.array(np.concatenate(little_X))

umap = UMAP()
umap_resp = umap.fit_transform(representation)

In [ ]:
plt.figure(figsize=(10,10))
plt.grid()
for i in range(10):
    plt.scatter(umap_resp[i * n:i * n + n, 0], umap_resp[i * n:i * n + n, 1], c='C' + str(i), label = int(i), s=.5)

plt.legend()

# Linear model

Your task is to train a linear classifier $\vec{x} \rightarrow y$ with SGD using TensorFlow.

You will need to calculate a logit (a linear transformation) $z_k$ for each class: 
$$z_k = \vec{x} \cdot \vec{w_k} + b_k \quad k = 0..9$$

And transform logits $z_k$ to valid probabilities $p_k$ with softmax: 
$$p_k = \frac{e^{z_k}}{\sum_{i=0}^{9}{e^{z_i}}} \quad k = 0..9$$

We will use a cross-entropy loss to train our multi-class classifier:
$$\text{cross-entropy}(y, p) = -\sum_{k=0}^{9}{\log(p_k)[y = k]}$$ 

where 
$$
[x]=\begin{cases}
       1, \quad \text{if $x$ is true} \\
       0, \quad \text{otherwise}
    \end{cases}
$$

Cross-entropy minimization pushes $p_k$ close to 1 when $y = k$, which is what we want.

Here's the plan:
* Flatten the images (28x28 -> 784) with `X_train.reshape((X_train.shape[0], -1))` to simplify our linear model implementation
* Use a matrix placeholder for flattened `X_train`
* Convert `y_train` to one-hot encoded vectors that are needed for cross-entropy
* Use a shared variable `W` for all weights (a column $\vec{w_k}$ per class) and `b` for all biases.
* Aim for ~0.93 validation accuracy

In [ ]:
# Higher-level API:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, hidden_size=40):
        super(Net, self).__init__()
        # here you construct weights for layers
        self.fc1 = nn.Linear(X_train.shape[1], hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, 10)
        
    def forward(self, x):
        # here you describe usage of layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        self.x = self.fc3(x)
        return F.log_softmax(self.x, dim=-1)

In [ ]:
# model interface:
model = Net()
tt = torch.from_numpy(X_train[:10, :].astype(np.float32))
output = model(tt)

print('Model outputs: \n', output)
# TODO: получите вероятности из output c помощью функций из torch
# hint: см документацию к log_softmax
probs = model.forward(tt)
print('Probs: \n', torch.exp(probs))

# TODO: получите предсказание из output c помощью функций из torch
pred = torch.argmax(probs, dim=1)
print('Pred: \n', pred.data.numpy())
print('Truth: \n', y_train[:10])

In [ ]:
from IPython.display import clear_output
from tqdm import trange

# функция для итераций по минибатчам, из первого семинара
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.random.permutation(len(inputs))
    for start_idx in trange(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [ ]:
def train(model, optimizer, batchsize=32):
    loss_log = []
    model.train()
    for x_batch, y_batch in iterate_minibatches(X_train, y_train, batchsize=batchsize, shuffle=True):
        # data preparation
        data = torch.from_numpy(x_batch.astype(np.float32))
        target = torch.from_numpy(y_batch.astype(np.int64))

        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        # compute gradients
        loss.backward()
        # make a step
        optimizer.step()
        loss = loss.item()
        loss_log.append(loss)
    return loss_log


def test(model):
    loss_log = []
    model.eval()
    
    data = torch.from_numpy(X_val.astype(np.float32))
    target = torch.from_numpy(y_val.astype(np.int64))
    
    output = model(data)
    loss = F.nll_loss(output, target)
    loss_log = loss.item()
    return loss_log

In [ ]:
def plot_history(train_history, val_history, title='loss'):
    if title == 'loss':
        clear_output()
    plt.figure(figsize=(10,10))
    plt.title(title)
    plt.plot(np.arange(len(train_history)), train_history, label='train')
    plt.plot((1 + np.arange(0, len(val_log))) * 1562, val_history, label='val', c='red')
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
train_log = []
val_log = []

model = Net()
opt = torch.optim.RMSprop(model.parameters(), lr=0.001)
batchsize = 32

for epoch in range(20):
    train_loss = train(model, opt, batchsize=batchsize)
    train_log.extend(train_loss)
    
    val_loss = np.mean(test(model))
    val_log.append(val_loss)    
    
    plot_history(train_log, val_log, title='loss')

In [ ]:
# TODO: добавьте подсчет точности
def train(model, optimizer, batchsize=32):
    loss_log, acc_log = [], []
    
    model.train()
    for x_batch, y_batch in iterate_minibatches(X_train, y_train, batchsize=batchsize, shuffle=True):
        # data preparation
        data = torch.from_numpy(x_batch.astype(np.float32))
        target = torch.from_numpy(y_batch.astype(np.int64))

        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        acc_loss = torch.mean((torch.argmax(output, dim=1) == target).float())
        # compute gradients
        loss.backward()
        # make a step
        optimizer.step()
        loss = loss.item()
        loss_log.append(loss)
        acc_log.append(acc_loss.item())
    
    return loss_log, acc_log


# TODO: добавьте подсчет точности:
def test(model):
    loss_log, acc_log = [], []
    model.eval()
    
    data = torch.from_numpy(X_val.astype(np.float32))
    target = torch.from_numpy(y_val.astype(np.int64))
    
    output = model(data)
    loss = F.nll_loss(output, target)
    acc_loss = acc_loss = torch.mean((torch.argmax(output, dim=1) == target).float())
    loss_log = loss.item()
    
    return loss_log, acc_loss





train_log, train_acc_log = [], []
val_log, val_acc_log = [], []

model = Net()
opt = torch.optim.RMSprop(model.parameters(), lr=0.001)
batchsize = 32


for epoch in range(10):
    # train
    train_loss, acc_los = train(model, opt, batchsize=batchsize)
    train_log.extend(train_loss)
    train_acc_log.extend(acc_los)
    
    # test
    val_loss, val_acc_loss = test(model)
    val_log.append(val_loss)
    val_acc_log.append(val_acc_loss)
    
    # store metrics
    
    
    # plot all metrics (loss and acc for train/val)
    plot_history(train_log, val_log, title='loss')
    plot_history(train_acc_log, val_acc_log, title='acc')
    

In [ ]:
n = 2000

little_X = []
for i in range(10):
    little_X.append(torch.Tensor(X_train[np.where(y_train == i)[0][: n]]))
    
model.eval()
model(torch.cat(little_X))
representation = model.x.detach().numpy()

umap = UMAP()
umap_resp = umap.fit_transform(representation)

In [ ]:
plt.figure(figsize=(10,10))
plt.grid()
for i in range(10):
    plt.scatter(umap_resp[i * n:i * n + n, 0], umap_resp[i * n:i * n + n, 1], c='C' + str(i), label = int(i), s=.5)

plt.legend()

# Simple CNN

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.features = nn.Sequential(
                      nn.Conv2d(1,10,3),
                      nn.MaxPool2d(3),
                      nn.ReLU(),
                      nn.Conv2d(10,16,3),
                      nn.MaxPool2d(3, 1),
                      nn.ReLU()
                    )
        self.classifier = nn.Sequential(
                      nn.Linear(4 * 4 * 16, 32),
                      nn.Linear(32, 16),
                      nn.Linear(16, 10)
                    )
    def forward(self, x):
        out = self.features(x.reshape((-1, 1, 28, 28)))
        self.out = self.classifier(out.reshape(-1, 4 * 16 * 4))
        return F.log_softmax(self.out, dim=-1)

In [ ]:
train_log = []
val_log = []

model = Net()
opt = torch.optim.RMSprop(model.parameters(), lr=0.001)
batchsize = 32

for epoch in range(20):
    train_loss = train(model, opt, batchsize=batchsize)
    train_log.extend(train_loss)
    
    val_loss = test(model)
    val_log.append(val_loss)
    
    plot_history(np.array(train_log)[0::2, :].flatten(), np.array(val_log)[:,0], title='loss')
    plot_history(np.array(train_log)[1::2, :].flatten(), np.array(val_log)[:,1], title='acc')

# Try to show data representation on your own

In [ ]:
n = 2000

little_X = []
for i in range(10):
    little_X.append(torch.Tensor(X_train[np.where(y_train == i)[0][: n]]))
    
model.eval()
model(torch.cat(little_X))
representation = model.x.detach().numpy()

umap = UMAP()


In [ ]:
### YOUR CODE HERE

In [ ]:
import sys
import os
import time

import numpy as np
from tqdm import trange
import matplotlib.pyplot as plt


def load_mnist(flatten=False):
    """taken from https://github.com/Lasagne/Lasagne/blob/master/examples/mnist.py"""
    # We first define a download function, supporting both Python 2 and 3.
    if sys.version_info[0] == 2:
        from urllib import urlretrieve
    else:
        from urllib.request import urlretrieve

    def download(filename, source='http://yann.lecun.com/exdb/mnist/'):
        print("Downloading %s" % filename)
        urlretrieve(source + filename, filename)

    # We then define functions for loading MNIST images and labels.
    # For convenience, they also download the requested files if needed.
    import gzip

    def load_mnist_images(filename):
        if not os.path.exists(filename):
            download(filename)
        # Read the inputs in Yann LeCun's binary format.
        with gzip.open(filename, 'rb') as f:
            data = np.frombuffer(f.read(), np.uint8, offset=16)
        # The inputs are vectors now, we reshape them to monochrome 2D images,
        # following the shape convention: (examples, channels, rows, columns)
        data = data.reshape(-1, 1, 28, 28)
        # The inputs come as bytes, we convert them to float32 in range [0,1].
        # (Actually to range [0, 255/256], for compatibility to the version
        # provided at http://deeplearning.net/data/mnist/mnist.pkl.gz.)
        return data / np.float32(256)

    def load_mnist_labels(filename):
        if not os.path.exists(filename):
            download(filename)
        # Read the labels in Yann LeCun's binary format.
        with gzip.open(filename, 'rb') as f:
            data = np.frombuffer(f.read(), np.uint8, offset=8)
        # The labels are vectors of integers now, that's exactly what we want.
        return data

    # We can now download and read the training and test set images and labels.
    X_train = load_mnist_images('train-images-idx3-ubyte.gz')
    y_train = load_mnist_labels('train-labels-idx1-ubyte.gz')
    X_test = load_mnist_images('t10k-images-idx3-ubyte.gz')
    y_test = load_mnist_labels('t10k-labels-idx1-ubyte.gz')

    # We reserve the last 10000 training examples for validation.
    X_train, X_val = X_train[:-10000], X_train[-10000:]
    y_train, y_val = y_train[:-10000], y_train[-10000:]

    if flatten:
        X_train = X_train.reshape([X_train.shape[0], -1])
        X_val = X_val.reshape([X_val.shape[0], -1])
        X_test = X_test.reshape([X_test.shape[0], -1])

    # We just return all the arrays in order, as expected in main().
    # (It doesn't matter how we do this as long as we can read them again.)
    return X_train, y_train, X_val, y_val, X_test, y_test


def plot_embedding(X, y):
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)

    plt.figure()
    ax = plt.subplot(111)
    for i in range(X.shape[0]):
        plt.text(X[i, 0], X[i, 1], str(y[i]),
                 color=plt.cm.Set1(y[i] / 10.),
                 fontdict={'weight': 'bold', 'size': 9})
    plt.xticks([]), plt.yticks([])
